In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.schema import SystemMessage, HumanMessage
import time
import random

# Use Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load source vectorstore (knowledge base)
source_vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

# Create destination vectorstore for storing synthetic Q&A pairs about architecture
architecture_qa_vectorstore = Chroma(persist_directory="./chroma_architecture_qa_db", embedding_function=embedding_model)

# Groq API key
groq_api_key = "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"

# Set up LLM for answer generation
llm = ChatGroq(model="llama3-8b-8192", groq_api_key=groq_api_key)

# Set up LLM for question generation
question_generator = ChatGroq(model="llama3-8b-8192", groq_api_key=groq_api_key)

# Define Retrieval-Augmented Generation (RAG) pipeline
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=source_vectorstore.as_retriever()
)

# List of architectural topics specific to Hampi
hampi_architectural_topics = [
    "Dravidian architectural style in Hampi temples",
    "Stone carving techniques used in Hampi monuments",
    "Pillared halls in Vijayanagara architecture",
    "Corbelled arches in Hampi structures",
    "Monolithic structures in Hampi",
    "Ornamental motifs in Hampi temple architecture",
    "Stepped tank designs in Hampi",
    "Mandapas in Vijayanagara temple architecture",
    "Gopurams of Hampi temples",
    "Stone aqueduct systems in Hampi",
    "Geometric patterns in Hampi temple carvings",
    "Structural engineering of Hampi monuments",
    "Temple layout patterns in Hampi",
    "Islamic influences on Vijayanagara architecture",
    "Vishnu and Shiva temple architectural differences in Hampi",
    "Astronomy-related architectural features in Hampi",
    "Sculptural narrative panels in Hampi temples",
    "Column designs and variations across Hampi",
    "Water management architectural features in Hampi",
    "Ceremonial gateway architecture in Hampi"
]

# List of architectural elements to incorporate into questions
architectural_elements = [
    "columns", "pillars", "capitals", "brackets", "ceilings", "domes", 
    "arches", "lintels", "cornices", "friezes", "pedestals", "plinths",
    "bases", "shafts", "entablatures", "moldings", "corbels", "buttresses",
    "vaults", "spires", "facades", "courtyards", "enclosures", "pavilions"
]

# Generate a specific architectural question about Hampi
def generate_architectural_question():
    # Randomly select topic and elements to focus on
    topic = random.choice(hampi_architectural_topics)
    element = random.choice(architectural_elements)
    
    # Create different types of architectural questions
    question_types = [
        f"What are the distinctive features of {element} in {topic}?",
        f"How does the design of {element} in Hampi compare with other South Indian architectural traditions?",
        f"What construction techniques were used for the {element} in {topic}?",
        f"What is the symbolic significance of the {element} designs in {topic}?",
        f"How did the {element} in {topic} evolve throughout the Vijayanagara period?",
        f"What materials were used in constructing the {element} in {topic}?",
        f"How did foreign influences affect the design of {element} in {topic}?",
        f"What are the ornamental patterns typically found on {element} in {topic}?",
        f"How does the layout of {element} in {topic} reflect astronomical knowledge?",
        f"What structural innovations can be seen in the {element} of {topic}?"
    ]
    
    question_template = random.choice(question_types)
    
    # For more natural variation, sometimes generate completely custom questions
    if random.random() < 0.3:
        prompt = f"""Create a specific, detailed question about architectural features, styles, or techniques of {topic} at Hampi.
        Focus specifically on construction methods, design elements, structural innovations, or artistic aspects of the architecture.
        The question should concentrate on architectural style only, not history or cultural significance.
        Provide ONLY the question with no additional text."""
        
        messages = [
            SystemMessage(content="You are an architectural historian specializing in Hampi and Vijayanagara architecture."),
            HumanMessage(content=prompt)
        ]
        
        response = question_generator.invoke(messages)
        return response.content.strip()
    
    return question_template

# Process a batch of synthetic architecture questions and store Q&A pairs
def process_architectural_qa_batch(num_questions=10):
    print(f"Generating {num_questions} synthetic architecture-focused Q&A pairs about Hampi...")
    
    for i in range(num_questions):
        # Generate an architecture-specific question
        question = generate_architectural_question()
        print(f"\nArchitectural Question {i+1}: {question}")
        
        # Generate an answer using the RAG pipeline
        try:
            answer = qa.run(question)
            print(f"Generated Answer: {answer[:100]}...")
            
            # Store the Q&A pair in the architecture vectorstore
            qa_text = f"Question: {question}\nAnswer: {answer}"
            
            # Extract the main topic for metadata
            topic_keywords = [topic for topic in hampi_architectural_topics if any(word in question.lower() for word in topic.lower().split())]
            topic = topic_keywords[0] if topic_keywords else "General Hampi architecture"
            
            architecture_qa_vectorstore.add_texts(
                [qa_text], 
                metadatas=[{
                    "question": question, 
                    "topic": topic,
                    "content_type": "architecture"
                }]
            )
            
            # Sleep briefly to avoid API rate limits
            time.sleep(1)
            
        except Exception as e:
            print(f"Error generating answer: {e}")
    
    # Persist the architecture QA vectorstore
    architecture_qa_vectorstore.persist()
    print(f"\nCompleted architectural Q&A generation. Stored {num_questions} pairs in ./chroma_architecture_qa_db")

# Function to search the architecture Q&A database
def search_architecture_qa_database(query, k=3):
    results = architecture_qa_vectorstore.similarity_search(query, k=k)
    return results

# Main execution
if __name__ == "__main__":
    # Ask how many synthetic architectural questions to generate
    try:
        num_questions = int(input("How many architectural Q&A pairs about Hampi would you like to generate? "))
    except ValueError:
        num_questions = 10
        print("Invalid input. Defaulting to 10 architectural questions.")
    
    # Generate synthetic architecture Q&A pairs
    process_architectural_qa_batch(num_questions)
    
    # Allow searching the new architecture Q&A database
    while True:
        search_query = input("\nEnter a query to search the architectural Q&A database (or 'exit' to quit): ")
        if search_query.lower() == 'exit':
            break
            
        results = search_architecture_qa_database(search_query)
        print("\nRelevant architectural Q&A pairs:")
        for i, doc in enumerate(results):
            print(f"\n--- Result {i+1} ---")
            print(doc.page_content)
            print("----------")

C:\Users\sameer\AppData\Local\Temp\ipykernel_31068\3031321793.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\sameer\AppData\Local\Temp\ipykernel_31068\3031321793.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  source_vectorstore = Chroma(persist_director

Generating 20 synthetic architecture-focused Q&A pairs about Hampi...

Architectural Question 1: What construction techniques were used for the moldings in Islamic influences on Vijayanagara architecture?


C:\Users\sameer\AppData\Local\Temp\ipykernel_31068\3031321793.py:115: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(question)


Generated Answer: I'm not familiar with the specific construction techniques used for moldings in Islamic influences o...

Architectural Question 2: What construction techniques were used for the spires in Ornamental motifs in Hampi temple architecture?
Generated Answer: The spires in Ornamental motifs in Hampi temple architecture were constructed using the technique of...

Architectural Question 3: What structural innovations can be seen in the courtyards of Geometric patterns in Hampi temple carvings?
Generated Answer: I'm not familiar with specific details about the courtyards of Geometric patterns in Hampi temple ca...

Architectural Question 4: How does the layout of plinths in Stone carving techniques used in Hampi monuments reflect astronomical knowledge?
Generated Answer: I don't know the answer....

Architectural Question 5: How does the layout of brackets in Stone carving techniques used in Hampi monuments reflect astronomical knowledge?
Generated Answer: I don't know the ans

C:\Users\sameer\AppData\Local\Temp\ipykernel_31068\3031321793.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  architecture_qa_vectorstore.persist()



Completed architectural Q&A generation. Stored 20 pairs in ./chroma_architecture_qa_db


In [1]:
# Function to save Q&A pairs in a TXT file
def save_qa_to_txt(qa_pairs, filename="Hampi_Architecture_QA.txt"):
    with open(filename, "w", encoding="utf-8") as file:
        file.write("Hampi Architectural Q&A\n")
        file.write("=" * 40 + "\n\n")

        for i, qa in enumerate(qa_pairs):
            file.write(f"Q{i+1}: {qa['question']}\n")
            file.write(f"A{i+1}: {qa['answer']}\n")
            file.write("-" * 40 + "\n\n")
    
    print(f"Q&A pairs saved to {filename}")

# Modify the process function to store Q&A results in a TXT file
def process_architectural_qa_batch_and_save(num_questions=10):
    print(f"Generating {num_questions} synthetic architecture-focused Q&A pairs about Hampi...")

    qa_pairs = []

    for i in range(num_questions):
        question = generate_architectural_question()
        print(f"\nArchitectural Question {i+1}: {question}")
        
        try:
            answer = qa.run(question)
            print(f"Generated Answer: {answer[:100]}...")

            qa_pairs.append({"question": question, "answer": answer})

            topic_keywords = [topic for topic in hampi_architectural_topics if any(word in question.lower() for word in topic.lower().split())]
            topic = topic_keywords[0] if topic_keywords else "General Hampi architecture"

            architecture_qa_vectorstore.add_texts(
                [f"Question: {question}\nAnswer: {answer}"],
                metadatas=[{"question": question, "topic": topic, "content_type": "architecture"}]
            )
            
            time.sleep(1)

        except Exception as e:
            print(f"Error generating answer: {e}")

    # Save results to a TXT file
    save_qa_to_txt(qa_pairs)

    # Persist the architecture QA vectorstore
    architecture_qa_vectorstore.persist()
    print(f"\nCompleted architectural Q&A generation. Stored {num_questions} pairs in ./chroma_architecture_qa_db")
